In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
from google.colab import files
uploaded = files.upload()



Saving Tweets.csv to Tweets.csv


In [4]:

import pandas as pd
import io

In [7]:
df = pd.read_csv((io.BytesIO(uploaded['Tweets.csv'])))

In [9]:
def normalizer(tweet):
      only_letters = re.sub("[^a-zA-Z]", " ", tweet)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [14]:
!pip install nltk
import nltk
nltk.download('all')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizerd
nltk.download('stopwords')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

ImportError: ignored

In [15]:
from nltk.corpus import wordnet as wn

In [16]:
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

In [17]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [18]:
train_x,val_x,train_y,val_y = train_test_split(x_vectorized,y)

In [19]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(train_x, train_y)

In [20]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(train_x, train_y)
model = gs_clf.best_estimator_

In [21]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [25]:
print(_statistics)

{'f1_score': 0.7912568306010929, 'confusion_matrix': array([[2042,  177,   76],
       [ 257,  460,   71],
       [ 101,   82,  394]]), 'precision': 0.7912568306010929, 'recall': 0.7912568306010929}


In [34]:
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])
model.predict(test_feature)

array(['negative'], dtype=object)

In [29]:
test_feature = vectorizer.transform(['Movie is great'])
model.predict(test_feature)

array(['positive'], dtype=object)

In [35]:
test_feature = vectorizer.transform(['I\'m okay'])
model.predict(test_feature)

array(['neutral'], dtype=object)

In [36]:
import pickle

In [37]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models'+".p", "wb"))